In [ ]:
# import os
# os.environ["GOOGLE_API_KEY"] = ''
# os.environ["LANGCHAIN_TRACING_V2"] = ""
# os.environ["LANGCHAIN_API_KEY"]=""

from dotenv import load_dotenv
load_dotenv()

## Prompts

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Answer the following question based only on the provided context:

<context>
{context}
</context>

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Answer the following question based only on the provided context:

<context>
{context}
</context>


Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Answer the following question based only on the provided context:

<context>
{context}
</context>

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Answer the following question based only on the provided context:

<context>
{context}
</context>

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
{
    "name": "physics", 
    "description": "Good for answering questions about physics", 
    "prompt_template": physics_template,
},
{
    "name": "math", 
    "description": "Good for answering math questions", 
    "prompt_template": math_template
},
{
    "name": "History", 
    "description": "Good for answering history questions", 
    "prompt_template": history_template
},
{
    "name": "computer science", 
    "description": "Good for answering computer science questions", 
    "prompt_template": computerscience_template
}]

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:

\```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
\```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

## LLM

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-pro", convert_system_message_to_human=True)
# llm = ChatOpenAI()

## Document

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector = Chroma.from_documents(documents, embedding_function)

## Destinations

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template(template=physics_template)

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vector.as_retriever()

physic_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
prompt = ChatPromptTemplate.from_template(template=computerscience_template)

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vector.as_retriever()

computerscience_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

general_chain = RunnablePassthrough.assign(
    answer=(ChatPromptTemplate.from_template("{history} {input}") | llm | StrOutputParser())
)

## Route to Destination

In [ ]:
def route(info):
    print(info)
    if info["destination"]:
        if "physics" in info["destination"].lower():
            return physic_chain
        elif "computer science" in info["destination"].lower():
            return computerscience_chain
    else:
        return general_chain

In [ ]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# destinations
# ['physics: Good for answering questions about physics',
#  'math: Good for answering math questions',
#  'History: Good for answering history questions',
#  'computer science: Good for answering computer science questions']

router_system_template = PromptTemplate.from_template(MULTI_PROMPT_ROUTER_TEMPLATE)
router_system_template = router_system_template.format(destinations=destinations_str)

router_system_template = HumanMessagePromptTemplate.from_template(router_system_template)

## Memory 

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder

memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables({})

In [ ]:
router_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="history", optional=True), # Put history into router llm ???
        router_system_template,
    ]
)

## Build Chain

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chains.router.llm_router import RouterOutputParser
from operator import itemgetter

router_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | router_prompt
    | llm 
    | RouterOutputParser()
)

In [ ]:

full_chain = (
    router_chain 
    | {"destination": itemgetter("destination"), "input": lambda x: x["next_inputs"]['input']} 
    | RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    ) 
    | RunnableLambda(route)
)

## Usage

In [ ]:
inputs = {"input": "how can software langsmith help with testing?"}
response = full_chain.invoke(inputs)
print(response)
memory.save_context(inputs, {"output": response.get("answer")})

In [ ]:
inputs = {"input": "sorry, what was I asking?"}
response = full_chain.invoke(inputs)
print(response)
print(response.get("answer"))

In [ ]:
full_chain.get_graph().print_ascii()

## API

In [1]:
from api.server import Server
from api.manager import Manager

server = Server("http://localhost:3000", "0929b6b4-1657-41db-a109-c33bc23abbb0")

manager = Manager(server)
manager.set_current_match(1)

match = manager.get_current_match()

print(match)
for msg in match.history_msgs:
    print(msg)

# api.send_message(1, "What's up, ethan?")
# api.get_match(1)

Match ID: 1, Name: testgame, Players: [<api.models.player.Player object at 0x000001402CE2F7F0>, <api.models.player.Player object at 0x000001402D00BB20>], History Messages: [<api.models.message.HistoryMessage object at 0x000001402D00BD60>, <api.models.message.HistoryMessage object at 0x000001402D00B760>, <api.models.message.HistoryMessage object at 0x000001402D00AE90>, <api.models.message.HistoryMessage object at 0x000001402D053790>]
Message ID: 1, Match ID: 1, Text: Hi, I am ethan, User ID: 1, Created At: 2024-04-03T06:36:05.273Z
Message ID: 2, Match ID: 1, Text: What's up, ethan?, User ID: 1, Created At: 2024-04-05T04:27:22.543Z
Message ID: 3, Match ID: 1, Text: What's up, ethan?, User ID: 1, Created At: 2024-04-05T04:33:03.564Z
Message ID: 4, Match ID: 1, Text: What's up, ethan?, User ID: 1, Created At: 2024-04-05T04:33:51.854Z
